In [1]:
import tensorflow as tf
from tensorflow import keras
from pydub import AudioSegment
from pathlib import PurePath
import pandas as pd
import numpy as np
import os
import random
from scipy.io import wavfile
import librosa
import io
import soundfile as sf

In [2]:
model = keras.models.load_model('./saved_models/8.keras')

In [3]:
dirs_in_testing = os.listdir('./dataset/testing/')
flac_filePaths = []
for el in dirs_in_testing[0:3]:
    for root, dirs, files in os.walk('./dataset/testing/' + el):
        for file in files:
            if file.endswith('.flac'):
                flac_filePaths.append(os.path.join(root, file))

flac_filePaths = [el.replace('\\', '/') for el in flac_filePaths]
flac_filePaths = [el.replace('.flac', '.wav') for el in flac_filePaths]

In [4]:
flac_filePaths[0:10]

['./dataset/testing/125/121124/125-121124-0000.wav',
 './dataset/testing/125/121124/125-121124-0001.wav',
 './dataset/testing/125/121124/125-121124-0002.wav',
 './dataset/testing/125/121124/125-121124-0003.wav',
 './dataset/testing/125/121124/125-121124-0004.wav',
 './dataset/testing/125/121124/125-121124-0005.wav',
 './dataset/testing/125/121124/125-121124-0006.wav',
 './dataset/testing/125/121124/125-121124-0007.wav',
 './dataset/testing/125/121124/125-121124-0008.wav',
 './dataset/testing/125/121124/125-121124-0009.wav']

In [5]:
mouthCues_main = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'X']

In [6]:
mouthCues = []
delta = 0.07
idx = random.randint(0, 10)
filePath = flac_filePaths[idx]
print(filePath)

audioData, sampleRate = librosa.load(filePath)
print(f"Duration: {len(audioData)/sampleRate} secs")
duration = len(audioData)/sampleRate

start = 0 # sec
end = delta # sec

i = 0
while(end <= duration):
    audioChunk = audioData[int(start*sampleRate):int(end*sampleRate)-1]
    mfccs = np.mean(librosa.feature.mfcc(y=audioChunk, sr=sampleRate, n_mfcc=20, n_fft=256, hop_length=64).T, axis=0)
    prediction = mouthCues_main[model.predict(mfccs.reshape(1, -1)).argmax(axis=1)[0]]
    mouthCues.append({"start": start, "end": end, "value": prediction})
    start = end
    end += (duration - end) if ((duration - end) < delta and (duration - end) > 0) else delta
    i += 1
print(i)

./dataset/testing/125/121124/125-121124-0004.wav
Duration: 14.88 secs


C:\Users\adity\AppData\Roaming\Python\Python311\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


1/1 [==============================] - 0s 50ms/step
213


In [13]:
mouthCues

[{'start': 0, 'end': 0.07, 'value': 'X'},
 {'start': 0.07, 'end': 0.14, 'value': 'X'},
 {'start': 0.14, 'end': 0.21000000000000002, 'value': 'X'},
 {'start': 0.21000000000000002, 'end': 0.28, 'value': 'X'},
 {'start': 0.28, 'end': 0.35000000000000003, 'value': 'X'},
 {'start': 0.35000000000000003, 'end': 0.42000000000000004, 'value': 'G'},
 {'start': 0.42000000000000004, 'end': 0.49000000000000005, 'value': 'B'},
 {'start': 0.49000000000000005, 'end': 0.56, 'value': 'C'},
 {'start': 0.56, 'end': 0.6300000000000001, 'value': 'F'},
 {'start': 0.6300000000000001, 'end': 0.7000000000000002, 'value': 'G'},
 {'start': 0.7000000000000002, 'end': 0.7700000000000002, 'value': 'B'},
 {'start': 0.7700000000000002, 'end': 0.8400000000000003, 'value': 'A'},
 {'start': 0.8400000000000003, 'end': 0.9100000000000004, 'value': 'C'},
 {'start': 0.9100000000000004, 'end': 0.9800000000000004, 'value': 'C'},
 {'start': 0.9800000000000004, 'end': 1.0500000000000005, 'value': 'C'},
 {'start': 1.0500000000000

In [14]:
finalRes = []
temp = '$'
start = 0
end = 0
for idx in range(0, len(mouthCues)):
    if(mouthCues[idx]['value'] != temp):
        if(temp != '$'):
            finalRes.append({"value": temp, "start": start, "end": end})
        start = mouthCues[idx]['start']
        end = mouthCues[idx]['end']
        temp = mouthCues[idx]['value']
    else:
        end = mouthCues[idx]['end']

finalRes.append({"value": temp, "start": start, "end": end})

In [15]:
finalRes

[{'value': 'X', 'start': 0, 'end': 0.35000000000000003},
 {'value': 'G', 'start': 0.35000000000000003, 'end': 0.42000000000000004},
 {'value': 'B', 'start': 0.42000000000000004, 'end': 0.49000000000000005},
 {'value': 'C', 'start': 0.49000000000000005, 'end': 0.56},
 {'value': 'F', 'start': 0.56, 'end': 0.6300000000000001},
 {'value': 'G', 'start': 0.6300000000000001, 'end': 0.7000000000000002},
 {'value': 'B', 'start': 0.7000000000000002, 'end': 0.7700000000000002},
 {'value': 'A', 'start': 0.7700000000000002, 'end': 0.8400000000000003},
 {'value': 'C', 'start': 0.8400000000000003, 'end': 1.0500000000000005},
 {'value': 'G', 'start': 1.0500000000000005, 'end': 1.1200000000000006},
 {'value': 'B', 'start': 1.1200000000000006, 'end': 1.3300000000000007},
 {'value': 'E', 'start': 1.3300000000000007, 'end': 1.4000000000000008},
 {'value': 'A', 'start': 1.4000000000000008, 'end': 1.4700000000000009},
 {'value': 'B', 'start': 1.4700000000000009, 'end': 1.680000000000001},
 {'value': 'X', 's

In [16]:
import json
with open(filePath.replace('.wav', '.json'), 'r') as fp:
    jsonData = json.load(fp)

In [17]:
jsonData['mouthCues']

[{'start': 0.0, 'end': 0.21, 'value': 'X'},
 {'start': 0.21, 'end': 0.42, 'value': 'B'},
 {'start': 0.42, 'end': 0.49, 'value': 'C'},
 {'start': 0.49, 'end': 0.84, 'value': 'B'},
 {'start': 0.84, 'end': 0.98, 'value': 'D'},
 {'start': 0.98, 'end': 1.19, 'value': 'B'},
 {'start': 1.19, 'end': 1.33, 'value': 'C'},
 {'start': 1.33, 'end': 1.4, 'value': 'B'},
 {'start': 1.4, 'end': 1.48, 'value': 'A'},
 {'start': 1.48, 'end': 1.72, 'value': 'B'},
 {'start': 1.72, 'end': 2.13, 'value': 'X'},
 {'start': 2.13, 'end': 2.2, 'value': 'C'},
 {'start': 2.2, 'end': 2.27, 'value': 'B'},
 {'start': 2.27, 'end': 2.41, 'value': 'C'},
 {'start': 2.41, 'end': 2.48, 'value': 'D'},
 {'start': 2.48, 'end': 2.55, 'value': 'B'},
 {'start': 2.55, 'end': 2.63, 'value': 'A'},
 {'start': 2.63, 'end': 2.81, 'value': 'B'},
 {'start': 2.81, 'end': 2.89, 'value': 'A'},
 {'start': 2.89, 'end': 2.99, 'value': 'C'},
 {'start': 2.99, 'end': 3.13, 'value': 'B'},
 {'start': 3.13, 'end': 3.2, 'value': 'G'},
 {'start': 3.2, 

In [12]:
def getMouthCues(filePath):
    mouthCues = []
    print(filePath)
    audioData, sampleRate = librosa.load(filePath)
    print(f"Duration: {len(audioData)/sampleRate} secs")
    duration = len(audioData)/sampleRate

    start = 0 # sec
    end = 0.05 # sec

    while(end <= duration):
        audioChunk = audioData[int(start*sampleRate):int(end*sampleRate)-1]
        mfccs = np.mean(librosa.feature.mfcc(y=audioChunk, sr=sampleRate, n_mfcc=90, n_fft=256, hop_length=64).T, axis=0)
        prediction = mouthCues_main[model.predict(mfccs.reshape(1, -1)).argmax(axis=1)[0]]
        mouthCues.append({"start": start, "end": end, "value": prediction})
        start = end
        end += 0.05
    
    finalRes = []
    start = mouthCues[0]['start']
    end = mouthCues[0]['end']
    for idx in range(1, len(mouthCues)):
        if(mouthCues[idx]['value'] != mouthCues[idx-1]['value']):
            # print(mouthCues[idx-1]['value'], start, end)
            finalRes.append({"value": mouthCues[idx-1]['value'], "start": start, "end": end})
            start = mouthCues[idx]['start']
            end = mouthCues[idx]['end']
        else:
            end = mouthCues[idx]['end']
    
    return finalRes